In [1]:
#載入資料
import pandas as pd

df = pd.read_csv('ChnSentiCorp_htl_all.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7766 entries, 0 to 7765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7766 non-null   int64 
 1   review  7765 non-null   object
dtypes: int64(1), object(1)
memory usage: 121.5+ KB


In [2]:
#清理資料
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7765 entries, 0 to 7765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7765 non-null   int64 
 1   review  7765 non-null   object
dtypes: int64(1), object(1)
memory usage: 182.0+ KB


In [3]:
#重新隨機排序DataFrame
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled.head()

,label,review
0,0,"真不知道那個酒店好在哪裡,很難找到,旁邊都在修路,遮擋住了酒店.進了房間,很驚訝,為什麼呢?..."
1,0,"位置一般,門口還在修路,沙塵滾滾,附近沒有像樣的食店,不建議入住"
2,0,豪華大床房房間大小一般，牆壁看起來很不舒服，都有掉落的斑駁痕跡，其他一般，在所有住過的經濟型...
3,1,交通倒是方便。價格很成問題，旁邊同類型、裝修差不多的酒店比這間要便宜（150元），可惜攜程沒...
4,1,房間還是一如既往的寬敞明亮，很舒適，在廣州市中心能有這樣大面積的客房很少，唯一不足是前臺CH...


In [4]:
#label欄位,0為差評,1為好評

df_shuffled['label'] = df_shuffled['label'].apply(lambda value: '好評' if value == 1 else '差評')
df_shuffled.head()

,label,review
0,差評,"真不知道那個酒店好在哪裡,很難找到,旁邊都在修路,遮擋住了酒店.進了房間,很驚訝,為什麼呢?..."
1,差評,"位置一般,門口還在修路,沙塵滾滾,附近沒有像樣的食店,不建議入住"
2,差評,豪華大床房房間大小一般，牆壁看起來很不舒服，都有掉落的斑駁痕跡，其他一般，在所有住過的經濟型...
3,好評,交通倒是方便。價格很成問題，旁邊同類型、裝修差不多的酒店比這間要便宜（150元），可惜攜程沒...
4,好評,房間還是一如既往的寬敞明亮，很舒適，在廣州市中心能有這樣大面積的客房很少，唯一不足是前臺CH...


In [19]:
len(df_shuffled)

7765

In [ ]:
#初始化chroma db
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import os
from dotenv import load_dotenv

load_dotenv()
#多筆資料和中文,最好直接使用huggingface提供的sentence embedding

# 初始化 ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")
hotel_info = client.get_or_create_collection(
    name='hotel_info',
)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/multilingual-e5-large")

for index, row in df_shuffled.iterrows():
    ids = []
    documents = []
    metadatas = []
    embeddings = []

    embedding = model.encode(row['review']).tolist()
    embeddings.append(embedding)

    ids.append(str(index))   
    documents.append(row['review'])
    metadatas.append({
        'message':row['label']
    })
    

    #插入資料
    hotel_info.add(
        ids = ids,
        documents = documents,
        embeddings=embeddings,
        metadatas = metadatas
    )
    
print("資料已經存入")

Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 4
Insert of existing embedding ID: 4
Add of existing embedding ID: 5
Insert of existing embedding ID: 5
Add of existing embedding ID: 6
Insert of existing embedding ID: 6
Add of existing embedding ID: 7
Insert of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 8
Add of existing embedding ID: 9
Insert of existing embedding ID: 9
Add of existing embedding ID: 10
Insert of existing embedding ID: 10
Add of existing embedding ID: 11
Insert of existing embedding ID: 11
Add of existing embedding ID: 12
Insert of existing embedding ID: 12
Add of existing embedding ID: 13
Insert of existing embedding ID: 13
Add of existing embedding ID: 14
Insert of existing em

7375
資料已經存入


In [11]:
hotel_info.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': array([[ 0.01103258,  0.00367525, -0.03003611, ...,  0.00422477,
         -0.03485984,  0.00144743],
        [ 0.0155171 , -0.00142099, -0.03764709, ...,  0.00976248,
         -0.03516549, -0.00204681],
        [ 0.00965643,  0.01150021, -0.03120336, ..., -0.01861285,
         -0.0447603 ,  0.00586364],
        ...,
        [ 0.02929634,  0.00075534, -0.03207712, ..., -0.01269605,
         -0.03243466, -0.00344312],
        [ 0.0191416 , -0.00532915, -0.02515149, ...,  0.00562914,
         -0.03561803,  0.01563948],
        [ 0.05620942,  0.00733784, -0.0323485 , ...,  0.00674176,
         -0.05438549,  0.01916638]], shape=(10, 1024)),
 'documents': ['真不知道那個酒店好在哪裡,很難找到,旁邊都在修路,遮擋住了酒店.進了房間,很驚訝,為什麼呢?房間不是一般的小,佈置也不是一般的奇怪,床竟然就在門的邊上.進了洗手間,更驚訝,是住過的酒店裡最小最小的衛生間了.比香港的酒店衛生間都小,洗澡很不爽,因為水也非常小.從視窗看出去,什麼風景都沒有.早上點了客房送餐,量極小,也不好吃.出門叫計程車更暈了,堂堂一個五星級酒店竟然等了二十多分鐘才等到計程車,上了車以後,旁邊一條路都在修,結果去機場飛機都沒趕上,太鬱悶了.以後不會去住這家破酒店了,怪不得住的人那麼少!',
  '位置一般,門口

In [ ]:
query_text = '門童和服務生都非常熱情'
embedding = model.encode(query_text).tolist()
hotel_info.query(
    query_embeddings=[embedding],
    n_results=10,
    where_document={'$contains':"門童"} ##document內一定要有包含門童
)

{'ids': [['6374',
   '5745',
   '4604',
   '4510',
   '478',
   '1478',
   '7',
   '753',
   '723',
   '72']],
 'embeddings': None,
 'documents': [['我的國外客人十分滿意，我本人對大堂和門童也是很滿意。',
   '房間設施很好，服務很周到．特別是門童和前臺，很熱情．酒店每晚都徵求意見，看有什麼不滿意的，這是其他酒店沒有的．',
   '第一感覺就是門童服務很到位，前臺服務也面帶微笑。房間比較寬敞明亮，上網速度也很快。很滿意的一家酒店！',
   '裝修較舊,特別是地毯的材質顏色顯得較髒,與四星的評級很不相稱,門童很熱情,贊一個',
   '房間、服務尚可，就是電視訊道太少，門童不是很積極的服務，可能天氣冷？',
   '門童好像不錯,其他不記得了晚上睡得還可以',
   '酒店的地理位置非常棒,住的高階商務間.感覺房間非常小.門童和服務生都非常熱情.還有免費的水果.地毯和裝修有些陳舊~',
   '房間設定不錯,感覺比較新.洗手間很寬敞明亮,美中不足是洗手間的地板,稍微沾上點水就容易滑倒.早餐不錯,品種比較多.服務員,門童都很熱情,總得來說比較滿意.',
   '房間很乾淨！早餐很豐盛！門童很好！已經是第二次訂那了。以後還會住。',
   '地段非常棒.住的是高階商務間.感覺房間有些小.另空調`的製冷效果也不行.不過門童和服務生很熱情.而且有免費水果.地毯和裝修有些陳舊~~~']],
 'uris': None,
 'data': None,
 'metadatas': [[{'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'},
   {'message': '好評'}]],
 'distances': [[0.20208819210529327,
   0.2243313789367675